# Create simple test function

In [1]:
import rioxarray
import xesmf as xe
import numpy as np
from osgeo import gdal
import xarray as xr
    
"""
Created on Sun Oct 26 2023

@author: Todd Clark
"""

import rioxarray
import xesmf as xe
import numpy as np
import numpy.matlib as npm
from osgeo import gdal
import rasterio

def replace_none_with_0(input_file_path, output_file_path):
    xr_array = rioxarray.open_rasterio(input_file_path)
    
    #replace possible null values
    xr_array = xr_array.where(xr_array != xr_array._FillValue, 0) #remove array null value
    xr_array = xr_array.where(xr_array.notnull(),0) #remove Nan, None, or NaT
    
    xr_array.rio.to_raster(output_file_path)

def rescale_resolution(input_file, output_file, scaling_factor):
    '''
    Parameters
    ----------
    input_file : string
        name of the file you want to rescale
    output_file : string
        name of file and location you want to rescale the file too
    scaling_factor : float
        example : 0.5 will make tiffs length and width half as man pixels rounded down

    Returns
    -------
    None.
    '''
    #open input file
    ds = gdal.Open(input_file)

    # Calculate the new dimensions
    new_width = int(ds.RasterXSize * scaling_factor)
    new_height = int(ds.RasterYSize * scaling_factor)

    # set options
    options = gdal.WarpOptions(
        format="GTiff",
        outputType = gdal.GDT_Float32, #outputType= gdal.GDT_Int64, this needs to be changed based on what is going to be done with the data for salt I am using a float because the data may have been fractionalized when being warped.
        width=new_width,
        height=new_height,
        resampleAlg=gdal.GRA_Average,  # You can change the resampling method as needed
    )
    
    #transfor,
    gdal.Warp(output_file, ds, options=options)

    ds = None  # Close the input file
        
def create_mask(input_file, output_file, threshold):
    '''
    creates mask with ones representing values over a threshold and zeros for values under and equal to the threshold
    Parameters
    ----------
    input_file : string
        location of file you want to make mask of
    output_file : string
        location that you want to save the mask to
    threshold : float
        minimum value to be set at one

    Returns
    -------
    None.

    '''
    ds = gdal.Open(input_file)

    # Read the data
    data = ds.ReadAsArray()

    # Create a binary mask based on the threshold
    mask = (data > threshold).astype(np.uint8)

    # Save the mask as a GeoTIFF file
    mask_driver = gdal.GetDriverByName('GTiff')
    mask_ds = mask_driver.Create(output_file, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Byte)
    mask_ds.GetRasterBand(1).WriteArray(mask)

    # Set the geo referencing information for the mask
    mask_ds.SetProjection(ds.GetProjection())
    mask_ds.SetGeoTransform(ds.GetGeoTransform())

    ds = None  # Close the input file
    mask_ds = None  # Close the mask file

def transform_5070_to_4326(tif_path_5070, tif_path_4326):
    '''
    takes tiff input in 5070 and transforms it to 4326 (I don't know what error is created here)

    Parameters
    ----------
    tif_path_5070 : string
        path to input 5070 tif file.
    tif_path_4326 : string
        path to output 4326 tif file
    Returns
    -------
    None.
    '''
    input_ds = gdal.Open(tif_path_5070)
    
        # set options
    options = gdal.WarpOptions(
        format="GTiff",
        outputType = gdal.GDT_Float32,
        srcSRS = "EPSG:5070", 
        dstSRS ="EPSG:4326",
        resampleAlg = gdal.GRA_Average
    )
    
    gdal.Warp(tif_path_4326, input_ds, options = options) # goes from srcSRS to dstSRS
    input_ds = None #close the data set

def convert_to_geochem_nc(tif_file_path, nc_output_path, template_ds, old_method = True):
    '''
    Parameters
    ----------
    tif_file_path : TYPE
        DESCRIPTION.
    nc_output_path : TYPE
        DESCRIPTION.
    template_ds : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    '''
    #open file as xarray object using rioxarray
    input_ds = rioxarray.open_rasterio(tif_file_path)
    
    #ensure there are no nan-values and replace all possible null values
    input_ds = input_ds.where(input_ds != input_ds._FillValue, 0) #remove fill values
    input_ds = input_ds.where(input_ds.notnull(),0) #remove Nan, None, or NaT
    
    #convert to data set
    input_ds = input_ds.to_dataset(name = "input")
    
    if(old_method):
        #rename axes so data will work with regridder
        input_ds = input_ds.rename({'x':'lon','y':'lat'})
        
        #create regrid object
        regridder = xe.Regridder(input_ds, template_ds, "conservative")
        
        #regrid xarray object
        regrided_ds = regridder(input_ds) 
        
        #remove average
        regrided_ds = regrided_ds * template_ds["AREA"] / 1e6 # the 1e6 is to convert from m^2 to km^2
        
        regrided_ds.to_netcdf(nc_output_path)
        
        return None
        
def tif_to_nc(fulrez_4326_path, lowrez_4326_path, output_nc_path, tif_5070_path, tif_5070_path_none_with_0, fulrez_4326_path_none_with_0, template_path,scaling_factor = 0.01):
    """
    Converts a TIFF file to a NetCDF file using a given template.

    Args:
        fulres_4326_path (str): The path to where the intermediate full-resolution TIFF will be stored in EPSG:4326 projection.
        lowres_4326_path (str): The path to where the intermediate low-resolution TIFF will be stored file in EPSG:4326 projection.
        output_nc_path (str): The path to save the output NetCDF file.
        tif_5070_path (str): The path to the input TIFF file in EPSG:5070 projection is stored.
        template_path (str): The path to the template NetCDF file.
        scaling_factor (float, optional): The scaling factor to apply during the resolution rescaling. Defaults to 0.01

    Returns:
        None

    Examples:
        tif_to_nc('fulres.tif', 'lowres.tif', 'result.nc', 'tif_5070.tif', 'template.nc', scaling_factor=0.1)
    """

    #define template
    template_ds=xr.open_dataset(template_path)
    
    #run methods
    replace_none_with_0(tif_5070_path, tif_5070_path_none_with_0)
    transform_5070_to_4326(tif_5070_path_none_with_0, fulrez_4326_path)
    replace_none_with_0(fulrez_4326_path, fulrez_4326_path_none_with_0)
    rescale_resolution(fulrez_4326_path_none_with_0, lowrez_4326_path, scaling_factor)
    convert_to_geochem_nc(lowrez_4326_path, output_nc_path, template_ds)


In [2]:
#from helper_functions import *
from osgeo import gdal
gdal.DontUseExceptions() #if you say UseExceptions() instead it makes it so you have to use a lower scaling_factor
import numpy as np
import pandas as pd
import rioxarray
import matplotlib.pyplot as plt

#from within the project
# from tif_to_nc import tif_to_nc
# from tif_to_nc import tif_to_nc_without_replacing_zeros

#TODO: I need to fix the sig-figs on range print outs

# create slice files
def slice_raster (input_file_path:str, output_folder,number_of_steps = "n/a", plot = False):
    
    #open array snd convert none values to zero
    xr_array = rioxarray.open_rasterio(input_file_path)
    xr_array = xr_array.where(xr_array != xr_array._FillValue,0)
    
    y_min = xr_array.y.min()
    y_max = xr_array.y.max()
    y_range = y_max - y_min
   
    if(number_of_steps == "n/a"):
        step_size = 500000 
    else:
        step_size = int(y_range/number_of_steps)
    
    slices = np.arange(y_min, y_max+1, step_size)
    
    slice_paths = []
    for index in range(len(slices)-1):
        
        slice_output_path = f"{output_folder}raster_fulrez_5070_{slices[index]:.0e}_to_{slices[index+1]:.0e}.tif"
        slice_paths.append(slice_output_path)
        
        xr_slice = xr_array.where(xr_array.y>slices[index]).where(xr_array.y<slices[index+1])
        xr_slice = xr_slice.where(xr_array.notnull(),0)
        xr_slice.rio.to_raster(slice_output_path)
        
        print(f"finished_{slices[index]:.0e}_to_{slices[index+1]:.0e}")
        if plot:
            plt.figure()
            xr_slice.plot()
            plt.title(f"{slices[0]:.0e}_to_{slices[index+1]:.0e}")
    
    return {"slice_indices": slices, "slice_paths": slice_paths}
    
# convert array of slice files
def convert_slices (
    slice_path_array, output_folder,slice_indices,
    scaling_factor = 0.024,
    template_path = r"template_data/dust_emissions_05.20210906.nc"):
    
    resolutions = ["fulrez", "lowres", "cf_nc"]
    epsg_s = [5070, 4326]
    
    slices = slice_indices
    
    output_nc_paths = []
    all_paths = []
    range_key = []
    
    for index, slice_path in enumerate(slice_path_array):
        
        slice_range = f'{slices[index]:.1e}_to_{slices[index+1]:.1e}'
        
        fulrez_4326_path = f"{output_folder}raster_{resolutions[0]}_{epsg_s[1]}_{slice_range}.tif"
        fulrez_4326_path_none_with_0 = f"{output_folder}raster_{resolutions[0]}_{epsg_s[1]}_{slice_range}_none_with_0.tif"
        lowrez_4326_path = f"{output_folder}raster_{resolutions[1]}_{epsg_s[1]}_{slice_range}.tif"
        output_nc_path = f"{output_folder}raster_{resolutions[2]}_{epsg_s[1]}_{slice_range}.nc"
        input_tif_5070_path = slice_path
        tif_5070_path_none_with_0 = f"{output_folder}raster_{resolutions[0]}_{epsg_s[0]}_{slice_range}_none_with_0.tif"
        
        tif_to_nc(
            fulrez_4326_path,
            lowrez_4326_path,
            output_nc_path,
            input_tif_5070_path,
            tif_5070_path_none_with_0,
            fulrez_4326_path_none_with_0,
            template_path,
            scaling_factor=scaling_factor)
        
        output_nc_paths.append(output_nc_path)
        all_paths.append([fulrez_4326_path, fulrez_4326_path_none_with_0, lowrez_4326_path, tif_5070_path_none_with_0, output_nc_path])
        range_key.append(slice_range)
        
        print(f"finished converting {slice_range}")
        
    return range_key, output_nc_paths, all_paths

def run_stats(keys, paths_to_rasters_2d):
    for index, key in enumerate(keys):
        paths_to_rasters_1d = paths_to_rasters_2d[index]
        slice_data = []
        index_names = []
        
        for path in paths_to_rasters_1d:
            xr_array = rioxarray.open_rasterio(path)
            xr_array = xr_array.where(xr_array != xr_array._FillValue,0) #remove none values so stats can calculate ok
            
            row_stats = {
                "Total_in_slice":float(xr_array.sum()),
                #"Average_Weight":, see teams
                "Lowest_lon":float(xr_array.x.min()),
                "Highest_lon":float(xr_array.x.max()),
                "Lowest_lat":float(xr_array.y.min()),
                "Highest_lat":float(xr_array.y.max()),
                "x_Resolution":np.abs(float(xr_array.x[1]) - float(xr_array.x[2])),
                "y_Resolution":np.abs(float(xr_array.y[1]) - float(xr_array.y[2]))
            }
            
            name = path[20:len(path)-4]
            slice_data.append(row_stats)
            index_names.append(name)
            print(f'    finished taking stats for {name}')
   
        slice_df = pd.DataFrame(slice_data, index = index_names)
        slice_df.to_csv(f"slice_stats/{key}.csv")
        print(f"finished taking stats for {key}")
    

# get stats

## Try to use alternate regrading method
    - Remember the method that we were trying to use #https://scitools-iris.readthedocs.io/en/v3.4.1/userguide/interpolation_and_regridding.html Area weighted regridding I don't think that this is very useful.
    - use the method

In [3]:
#Make area grid so I can calculate switch mean re-gridding to sum re-gridding
# original kg/km^3 - > 
# input_raster = 

In [5]:
input_tif_5070_path = r"input_data/1992_2015/1993.tif"
template_path = r"template_data/dust_emissions_05.20210906.nc"
scaling_factor = 0.024

output_dict = slice_raster(input_tif_5070_path,r'slicer_test/',number_of_steps=10, plot = False)
print(output_dict)
slice_indices = output_dict["slice_indices"]
slice_paths = output_dict['slice_paths']

conversion_data = convert_slices(slice_paths, r'convert_slices_test/', slice_indices)
# {'slice_indices': np.array([      0,  500000, 1000000, 1500000, 2000000, 2500000, 3000000,
#        3500000]), 'slice_paths': ['slicer_test/raster_fulrez_5070_0e+00_to_5e+05.tif', 'slicer_test/raster_fulrez_5070_5e+05_to_1e+06.tif', 'slicer_test/raster_fulrez_5070_1e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_3e+06.tif', 'slicer_test/raster_fulrez_5070_3e+06_to_4e+06.tif']}
#

#conversion_data = (['0.0e+00_to_5.0e+05', '5.0e+05_to_1.0e+06', '1.0e+06_to_1.5e+06', '1.5e+06_to_2.0e+06', '2.0e+06_to_2.5e+06', '2.5e+06_to_3.0e+06', '3.0e+06_to_3.5e+06'], ['convert_slices_test/raster_cf_nc_4326_0.0e+00_to_5.0e+05.nc', 'convert_slices_test/raster_cf_nc_4326_5.0e+05_to_1.0e+06.nc', 'convert_slices_test/raster_cf_nc_4326_1.0e+06_to_1.5e+06.nc', 'convert_slices_test/raster_cf_nc_4326_1.5e+06_to_2.0e+06.nc', 'convert_slices_test/raster_cf_nc_4326_2.0e+06_to_2.5e+06.nc', 'convert_slices_test/raster_cf_nc_4326_2.5e+06_to_3.0e+06.nc', 'convert_slices_test/raster_cf_nc_4326_3.0e+06_to_3.5e+06.nc'], [['convert_slices_test/raster_fulrez_4326_0.0e+00_to_5.0e+05.tif', 'convert_slices_test/raster_fulrez_4326_0.0e+00_to_5.0e+05_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_0.0e+00_to_5.0e+05.tif', 'convert_slices_test/raster_fulrez_5070_0.0e+00_to_5.0e+05_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_0.0e+00_to_5.0e+05.nc'], ['convert_slices_test/raster_fulrez_4326_5.0e+05_to_1.0e+06.tif', 'convert_slices_test/raster_fulrez_4326_5.0e+05_to_1.0e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_5.0e+05_to_1.0e+06.tif', 'convert_slices_test/raster_fulrez_5070_5.0e+05_to_1.0e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_5.0e+05_to_1.0e+06.nc'], ['convert_slices_test/raster_fulrez_4326_1.0e+06_to_1.5e+06.tif', 'convert_slices_test/raster_fulrez_4326_1.0e+06_to_1.5e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_1.0e+06_to_1.5e+06.tif', 'convert_slices_test/raster_fulrez_5070_1.0e+06_to_1.5e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_1.0e+06_to_1.5e+06.nc'], ['convert_slices_test/raster_fulrez_4326_1.5e+06_to_2.0e+06.tif', 'convert_slices_test/raster_fulrez_4326_1.5e+06_to_2.0e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_1.5e+06_to_2.0e+06.tif', 'convert_slices_test/raster_fulrez_5070_1.5e+06_to_2.0e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_1.5e+06_to_2.0e+06.nc'], ['convert_slices_test/raster_fulrez_4326_2.0e+06_to_2.5e+06.tif', 'convert_slices_test/raster_fulrez_4326_2.0e+06_to_2.5e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_2.0e+06_to_2.5e+06.tif', 'convert_slices_test/raster_fulrez_5070_2.0e+06_to_2.5e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_2.0e+06_to_2.5e+06.nc'], ['convert_slices_test/raster_fulrez_4326_2.5e+06_to_3.0e+06.tif', 'convert_slices_test/raster_fulrez_4326_2.5e+06_to_3.0e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_2.5e+06_to_3.0e+06.tif', 'convert_slices_test/raster_fulrez_5070_2.5e+06_to_3.0e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_2.5e+06_to_3.0e+06.nc'], ['convert_slices_test/raster_fulrez_4326_3.0e+06_to_3.5e+06.tif', 'convert_slices_test/raster_fulrez_4326_3.0e+06_to_3.5e+06_none_with_0.tif', 'convert_slices_test/raster_lowres_4326_3.0e+06_to_3.5e+06.tif', 'convert_slices_test/raster_fulrez_5070_3.0e+06_to_3.5e+06_none_with_0.tif', 'convert_slices_test/raster_cf_nc_4326_3.0e+06_to_3.5e+06.nc']])

print(conversion_data)

keys, nc_paths, all_paths = conversion_data

run_stats(keys, all_paths)

finished_2e+05_to_5e+05
finished_5e+05_to_8e+05
finished_8e+05_to_1e+06
finished_1e+06_to_1e+06
finished_1e+06_to_2e+06
finished_2e+06_to_2e+06
finished_2e+06_to_2e+06
finished_2e+06_to_3e+06
finished_3e+06_to_3e+06
finished_3e+06_to_3e+06
{'slice_indices': array([ 177785.,  490985.,  804185., 1117385., 1430585., 1743785.,
       2056985., 2370185., 2683385., 2996585., 3309785.]), 'slice_paths': ['slicer_test/raster_fulrez_5070_2e+05_to_5e+05.tif', 'slicer_test/raster_fulrez_5070_5e+05_to_8e+05.tif', 'slicer_test/raster_fulrez_5070_8e+05_to_1e+06.tif', 'slicer_test/raster_fulrez_5070_1e+06_to_1e+06.tif', 'slicer_test/raster_fulrez_5070_1e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_2e+06.tif', 'slicer_test/raster_fulrez_5070_2e+06_to_3e+06.tif', 'slicer_test/raster_fulrez_5070_3e+06_to_3e+06.tif', 'slicer_test/raster_fulrez_5070_3e+06_to_3e+06.tif']}


c:\Users\theto\miniforge3\envs\geospatial\Lib\site-packages\xesmf\backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


finished converting 1.8e+05_to_4.9e+05


c:\Users\theto\miniforge3\envs\geospatial\Lib\site-packages\xesmf\backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


finished converting 4.9e+05_to_8.0e+05


...

finished converting 2.4e+06_to_2.7e+06


c:\Users\theto\miniforge3\envs\geospatial\Lib\site-packages\xesmf\backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


finished converting 2.7e+06_to_3.0e+06


c:\Users\theto\miniforge3\envs\geospatial\Lib\site-packages\xesmf\backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


finished converting 3.0e+06_to_3.3e+06
(['1.8e+05_to_4.9e+05', '4.9e+05_to_8.0e+05', '8.0e+05_to_1.1e+06', '1.1e+06_to_1.4e+06', '1.4e+06_to_1.7e+06', '1.7e+06_to_2.1e+06', '2.1e+06_to_2.4e+06', '2.4e+06_to_2.7e+06', '2.7e+06_to_3.0e+06', '3.0e+06_to_3.3e+06'], ['convert_slices_test/raster_cf_nc_4326_1.8e+05_to_4.9e+05.nc', 'convert_slices_test/raster_cf_nc_4326_4.9e+05_to_8.0e+05.nc', 'convert_slices_test/raster_cf_nc_4326_8.0e+05_to_1.1e+06.nc', 'convert_slices_test/raster_cf_nc_4326_1.1e+06_to_1.4e+06.nc', 'convert_slices_test/raster_cf_nc_4326_1.4e+06_to_1.7e+06.nc', 'convert_slices_test/raster_cf_nc_4326_1.7e+06_to_2.1e+06.nc', 'convert_slices_test/raster_cf_nc_4326_2.1e+06_to_2.4e+06.nc', 'convert_slices_test/raster_cf_nc_4326_2.4e+06_to_2.7e+06.nc', 'convert_slices_test/raster_cf_nc_4326_2.7e+06_to_3.0e+06.nc', 'convert_slices_test/raster_cf_nc_4326_3.0e+06_to_3.3e+06.nc'], [['convert_slices_test/raster_fulrez_4326_1.8e+05_to_4.9e+05.tif', 'convert_slices_test/raster_fulrez_4326